#PREPARE DATA

In [ ]:
!pip install langchain_community
!pip install langchain
!pip install bs4
!pip install fpdf
!pip install requests
!pip install gradio

In [ ]:
import os
import re
import json
from langchain_community.document_loaders import RecursiveUrlLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup
import requests

In [ ]:
from posixpath import split
import os
import json
import re
import requests
from bs4 import BeautifulSoup
from langchain.document_loaders import WebBaseLoader, RecursiveUrlLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Hàm trích xuất và làm sạch nội dung từ HTML
def extract_and_clean(html: str) -> str:
    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    content = soup.get_text()

    # Clean the text: giữ lại các ký tự a-z, A-Z, 0-9, dấu ',' và '.'
    cleaned_content = re.sub(r'[^a-zA-Z0-9,.\s]', '', content)  # Loại bỏ ký tự đặc biệt
    # Xóa các khoảng trắng dư thừa (nhiều dấu cách liền nhau, đầu và cuối)
    cleaned_content = re.sub(r'\s+', ' ', cleaned_content).strip()
    return cleaned_content

# Hàm crawl dữ liệu từ URL với RecursiveUrlLoader
def crawl_data(url_data):
    loader = RecursiveUrlLoader(url_data, extractor=extract_and_clean, max_depth=6)  # Increase depth if needed
    docs = loader.load()
    print('length:', len(docs))

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_documents(docs)
    print('length_chunks:', len(chunks))  # In số lượng đoạn văn bản sau khi chia
    return chunks

# Hàm crawl dữ liệu web trực tiếp với WebBaseLoader
def crawl_data_web(url_data):
    session = requests.Session()
    session.verify = False  # Vô hiệu hóa kiểm tra SSL
    # Set headers to mimic a browser
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
    })
    loader = WebBaseLoader(url_data, session=session)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    docs = text_splitter.split_documents(docs)
    print('length:', len(docs))
    return docs

def save_data(documents, filename, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory, filename)

    # Chuyển đổi documents thành định dạng có thể serialize
    data_to_save = [{'page_content': doc.page_content, 'metadata': doc.metadata} for doc in documents]

    # Lưu vào file JSON với encoding UTF-8
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data_to_save, file, ensure_ascii=False, indent=4)
    print(f'Data saved to {file_path}')


In [ ]:
links_web = '/content/web_links.txt'

def get_links_from_file(file_path):
    with open(file_path, 'r') as file:
        links = file.read().splitlines()
    return links

print(get_links_from_file(links_web))
print(len(get_links_from_file(links_web)))

['https://uet.vnu.edu.vn/lich-su/', 'https://uet.vnu.edu.vn/chien-luoc-phat-trien/', 'https://uet.vnu.edu.vn/co-cau-to-chuc/', 'https://uet.vnu.edu.vn/nha-giao-nhan-dan-nha-giao-uu-tu/', 'https://uet.vnu.edu.vn/doi-ngu-can-bo/', 'https://uet.vnu.edu.vn/co-so-dao-tao/', 'https://uet.vnu.edu.vn/dao-tao-dai-hoc/', 'https://uet.vnu.edu.vn/dao-tao-sau-dai-hoc/', 'https://uet.vnu.edu.vn/dinh-muc-hoc-phi-dao-tao-dai-hoc-nam-hoc-2024-2025/', 'https://tuyensinh.uet.vnu.edu.vn/category/phuong-thuc-xet-tuyen/xet-tuyen-thang/', 'https://tuyensinh.uet.vnu.edu.vn/category/phuong-thuc-xet-tuyen/uu-tien-xet-tuyen/', 'https://tuyensinh.uet.vnu.edu.vn/category/phuong-thuc-xet-tuyen/sat-alevel-quocte/', 'https://tuyensinh.uet.vnu.edu.vn/category/nganh-theo-khoa/vien-tri-tue-nhan-tao/', 'https://tuyensinh.uet.vnu.edu.vn/category/nganh-theo-khoa/khoa-cong-nghe-thong-tin/', 'https://tuyensinh.uet.vnu.edu.vn/category/nganh-theo-khoa/khoa-dien-tu-vien-thong/', 'https://tuyensinh.uet.vnu.edu.vn/category/nganh-

In [ ]:
links_web = get_links_from_file(links_web)
data_web = []
#crawl data from links
for link in links_web:
  data_web.extend(crawl_data_web(link))
#save data to json file
save_data(data_web, 'data_web.json', 'data')

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'uet.vnu.edu.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


length: 42


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'uet.vnu.edu.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


length: 32


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'uet.vnu.edu.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

length: 3


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'tuyensinh.uet.vnu.edu.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


length: 3


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'uet.vnu.edu.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


length: 15
length: 4
length: 2
length: 1
length: 6
length: 2
length: 3
length: 5
Data saved to data/data_web.json


In [ ]:
print(len(data_web))

252


#Xử lý dữ liệu

In [ ]:
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.7 MB/s eta 0:00:00


In [ ]:
import re

def flatten_content(data):
    # Trích xuất page_content và metadata
    page_content = data.get("page_content", "")
    metadata = data.get("metadata", {})

    # Làm phẳng nội dung
    flat_content = re.sub(r'\n+', '\n', page_content).strip()  # Loại bỏ dòng trống
    flat_content = re.sub(r'\s+', ' ', flat_content)  # Thay thế nhiều khoảng trắng bằng một khoảng trắng

    # Lấy thông tin tiêu đề và nguồn từ metadata
    title = metadata.get("title", "No Title")
    source = metadata.get("source", "No Source")

    # Kết hợp chỉ giữ lại content, source
    result = f"Content:{flat_content}|Source:{source}"
    return result

def word_segmentation(text):
    from underthesea import word_tokenize
    return word_tokenize(text, format="text")
processed_data = []
with open('data/data_web.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
    for item in data:
        processed_data.append(word_segmentation(flatten_content(item)))

# Lưu vào file
with open('data/processed_data.json', 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)


#PREPARE DB

In [ ]:
!pip install faiss-cpu
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.5 MB/s eta 0:00:00


In [ ]:
import json
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
data_path = '/content/data/processed_data.json'
vector_db_path = 'vector_db/'
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

def create_db(persist_directory):
    if not os.path.exists(persist_directory):
        os.makedirs(persist_directory)

    with open(data_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
    db = FAISS.from_texts(data, embeddings)
    db.save_local(persist_directory)
    return db

def load_db(persist_directory):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = FAISS.load_local(persist_directory, embeddings, allow_dangerous_deserialization=True)
    return vector_db

create_db(vector_db_path)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#LOAD LLM

In [ ]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
GROQ = userdata.get('GROQ_API')

In [ ]:
import os
os.environ['GROQ_API_KEY'] = GROQ

In [104]:
from langchain_groq import ChatGroq
def load_llm():
  llm = ChatGroq(
      groq_api_key = GROQ,
      model_name = 'Llama3-70b-8192',
      temperature=0.3,
      top_p = 0.5,
      max_tokens = 1024,
  )
  return llm

#Set Retriever

In [113]:
!pip install langchain_core
!pip install rank_bm25

In [114]:
from langchain.tools.retriever import create_retriever_tool
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever
from langchain_core.documents import Document

In [115]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS

def create_retriever(db) -> EnsembleRetriever:
    FAISS_retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 10},
    )

    documents_content = [doc.page_content for doc in db.similarity_search("", k=100)]
    bm25_retriever = BM25Retriever.from_texts(documents_content)
    bm25_retriever.k = 10

    ensemble_retriever = EnsembleRetriever(
        retrievers=[FAISS_retriever, bm25_retriever],
        weights=[0.8, 0.2],
    )
    return ensemble_retriever

#QA

In [116]:
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [117]:
def create_prompt(template):
  prompt = PromptTemplate(template=template, input_variables=["context","question"])
  return prompt

def create_chain(llm, prompt, db):
  retriever = create_retriever(db)
  chain = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type='stuff',
      retriever=retriever,
      return_source_documents=True,
      chain_type_kwargs={'prompt': prompt}
  )
  return chain

def get_answer(chain, question):
  res = chain.invoke({'query': question})
  return res


def get_completion(res):
  ques = res['query']
  ans = res['result']
  print(f'Question: {ques}')
  print(f'Answer: {ans}')

In [122]:
db = load_db(vector_db_path)
llm = load_llm()

template = '''
<|im_start|>system
Bạn là trợ lý AI thông minh, chuyên hỗ trợ sinh viên của Đại học Công nghệ. Khi trả lời, hãy cung cấp thông tin chi tiết, chính xác và dễ hiểu, nhưng chỉ giới hạn trong những nội dung có liên quan đến trường Đại học Công nghệ. Hãy giữ câu trả lời ngắn gọn, không đi sâu vào các nội dung ngoài phạm vi của trường. Nếu câu hỏi nằm ngoài phạm vi thông tin của trường, lịch sự từ chối trả lời.

Khi trả lời, vui lòng thực hiện các bước sau:
1. Trích xuất các chi tiết quan trọng từ thông tin tham khảo dưới đây nếu có.
2. Từ những chi tiết quan trọng đó đối chiếu với câu hỏi đưa ra bởi sinh viên.
3. Trả lời câu hỏi một cách chính xác, chỉ tập trung vào nội dung có liên quan đến trường.
4. Dịch câu trả lời sang Tiếng Việt và đảm bảo rõ ràng, dễ hiểu cho sinh viên.

Thông tin tham khảo:
{context}
<|im_end|>
<|im_start|>user
Câu hỏi của sinh viên:
{question}
<|im_end|>
<|im_start|>assistant
Câu trả lời (liên quan đến Đại học Công nghệ):
'''



prompt = create_prompt(template)
chain = create_chain(llm, prompt, db)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_groq/chat_models.py:362: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


In [124]:
question = 'sinh viên CLC có bắt buộc phải hoàn thành môn tiếng Anh trong năm 3 không ạ?'
res = get_answer(chain, question)
get_completion(res)

Question: sinh viên CLC có bắt buộc phải hoàn thành môn tiếng Anh trong năm 3 không ạ?
Answer: Không có thông tin cụ thể về yêu cầu hoàn thành môn tiếng Anh trong năm 3 cho sinh viên CLC tại Đại học Công nghệ. Tuy nhiên, theo thông tin chung, sinh viên tại Đại học Công nghệ cần đáp ứng yêu cầu về tiếng Anh để tốt nghiệp. Bạn nên tham khảo thêm thông tin từ phòng đào tạo hoặc giáo viên hướng dẫn của mình để biết thêm chi tiết về yêu cầu này.


#APP

In [121]:
import gradio as gr

def answer_question(question):
    res = get_answer(chain, word_segmentation(question))
    answer = res['result']
    return answer
with gr.Blocks() as rag_interface:
    gr.Markdown("<h1>University FAQ Helper</h1><p>Ask a question, and I’ll retrieve relevant information to provide an answer!</p>")

    # Input for the user's question
    question_input = gr.Textbox(label="Nhập câu hỏi của bạn", placeholder="Type your question here...", lines=2)

    # Output for the assistant's response
    answer_output = gr.Textbox(label="Câu trả lời", placeholder="Answer will appear here...", lines=6, interactive=False)

    # Define the submit button and link it to `answer_question`
    submit_button = gr.Button("Submit")
    submit_button.click(fn=answer_question, inputs=question_input, outputs=answer_output)

# Launch the interface
rag_interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d8087d3ef81d3bf98a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
